# ONS Census 2021 - PII Risk

### Importing/Cleaning Data

In [ ]:
import os, pathlib
import pandas as pd
import warnings

root = pathlib.Path("data")

region_sizes = {"Country" : "ctry",
                "Region" : "rgn",
                "Upper-Tier Local Authority" : "utla",      #153 in England
                "Lower-Tier Local Authority" : "ltla",      #296 in England
                "Middle-Layer Super Output Area" : "msoa",  #2000-6000 households; 5000-15000 persons
                "Lower-Layer Super Output Area" : "lsoa",   #400-1200 households; 1000-3000 persons
                "Output Area" : "oa"}                       #40-250 households; 100-625 persons

region_indices = {3 : "ctry", #how many are per country
                  10 : "rgn",
                  174 : "utla",
                  331 : "ltla",
                  7264 : "msoa",
                  35672 : "lsoa",
                  188800 : "oa"}

regions = list(region_sizes.values())

def import_data(region : str, target_groups : list = None, target_codes: list = None): 
    '''Imports target census data\n
    Parameters:\n
        region, should be string of one of the following: ctry, rgn, utla, ltla, msoa, lsoa, oa\n
        target_groups, should be a list of strings of the descriptions of the data values as found at https://www.nomisweb.co.uk/census/2021/bulk\n
        target_codes, should be a list of strings of the codes of the data values as found at https://www.nomisweb.co.uk/census/2021/bulk\n
    Note: one of the two target parameters must be passed\n

    Returns dictionary of "Data type code" : Pandas DataFrame'''
    if target_groups == None and target_codes == None:
        raise ValueError("Need specified groups to import")
    
    region = str(region)
    if region not in regions:
        regions_error = '\n'.join(f"{key}  :  {val}" for key, val in region_sizes.items())
        raise ValueError(f"Region code must be one of the following: \nRegion  :  Region Code\n{regions_error}")
    
    codes = pd.read_csv(root / "census_codes.csv")
    codes["Filename"] = (
    codes["Filename"].str.split(".")
    .apply(lambda x: x[0])
    )

    if target_groups:
        valid_target_groups = [group for group in target_groups if group in codes["Description"].values]
        if len(valid_target_groups) == 0:
            raise ValueError("Inputs not found in code database, ensure the group names are as found on https://www.nomisweb.co.uk/census/2021/bulk")
        erroneous_groups = [group for group in target_groups if group not in valid_target_groups]
        output_erroneous_groups = '\n'.join(erroneous_groups)
        if len(erroneous_groups) > 0:
            warnings.warn(f"The following groups were not imported as they are invalid: {output_erroneous_groups}\nEnsure the group names are as found on https://www.nomisweb.co.uk/census/2021/bulk")
        
    if target_codes:
        valid_target_codes = [code for code in target_codes if code in codes["Code"].values]
        if len(valid_target_codes) == 0:
            raise ValueError("Inputs not found in code database, ensure codes are in the form 'TSXXX' where X's are digits, as found on https://www.nomisweb.co.uk/census/2021/bulk")
        erroneous_codes = [code for code in target_codes if code not in valid_target_codes]
        output_erroneous_codes = '\n'.join(erroneous_codes)
        if len(erroneous_codes) > 0:
            warnings.warn(f"The following codes were not imported as they are invalid: {output_erroneous_codes}\nEnsure codes are in the form 'TSXXX' where X's are digits, as found on https://www.nomisweb.co.uk/census/2021/bulk")
    
    if target_groups:
        try:
            valid_target_codes = valid_target_codes
        except UnboundLocalError:
            valid_target_codes = []
        for group in valid_target_groups:
            valid_target_codes.append(codes.loc[codes["Description"] == group,"Code"].item())

    data = {}
    for code in valid_target_codes:
        folder = codes.loc[codes["Code"] == code, "Filename"].item()
        try:
            data.update({code : pd.read_csv(root / folder / f"{folder}-{region}.csv").drop(columns="date")})
        except FileNotFoundError:
            warnings.warn(f"File {folder}-{region}.csv not found in data/{folder}")
    
    return data

def import_all_data(region : str):
    '''Imports all installed census data\n
    Parameters:\n
        region, should be string of one of the following: ctry, rgn, utla, ltla, msoa, lsoa, oa\n
    Returns dictionary of "Data type code" : Pandas DataFrame
    '''
    region = str(region)
    if region not in regions:
        regions_error = '\n'.join(f"{key}  :  {val}" for key, val in region_sizes.items())
        raise ValueError(f"Region code must be one of the following: \nRegion  :  Region Code\n{regions_error}")

    codes = pd.read_csv(root / "census_codes.csv")
    codes["Filename"] = (
    codes["Filename"].str.split(".")
    .apply(lambda x: x[0])
    )
    
    data = {}
    for code in codes["Code"].values:
        folder = codes.loc[codes["Code"] == code, "Filename"].item()
        try:
            data.update({code : pd.read_csv(root / folder / f"{folder}-{region}.csv").drop(columns="date")})
        except FileNotFoundError:
            warnings.warn(f"File {folder}-{region}.csv not found in data/{folder}")
    
    return data

def cleanup_all(data : dict, remove_geography : bool = True, remove_geography_code : bool = True):
    '''
    Removes the specified columns from all dataframes\n
    Parameters:\n
        data, data to be cleaned, dictionary of "Data type code" : Pandas DataFrame\n
        remove_geography, whether to remove geography column, bool\n
        remove_geography-code, whether to remove geography code column, bool\n
        
    Returns dictionary of "Data type code" : Pandas DataFrame
    '''
    for key in data.keys():
        dataframe = data[key]
        try:
            if remove_geography:
                dataframe = dataframe.drop(columns="geography")
        except KeyError:
            pass
        try:
            if remove_geography_code:
                dataframe = dataframe.drop(columns="geography code")
        except KeyError:
            pass
        data[key] = dataframe
    return data

def cleanup(dataframe, columns : list):
    '''Removes specified columns from dataframe\n
    Parameters:\n
        dataframe, Pandas DataFrame to be cleaned\n
        columns, list of strings of column names to be removed\n
    '''
    if type(columns) is not list:
        raise ValueError("Parameter 'columns' must be a list")
    elif len([column for column in columns if type(column) is str]) == 0:
        raise ValueError("Parameter 'columns' must be a list of strings")  
    for column in columns:
        try:
            dataframe = dataframe.drop(columns=column)
        except KeyError:
            warnings.warn(f"Column {column} does not exist ")
    return dataframe

def factor_in_age(df, name=None):
    df_age = import_data(region_indices[len(df.index)], target_codes=["TS004"])["TS004"]
    df_age_totals_column = [column for column in list(df_age.columns) if "Total" in column][0]
    df_totals_column = [column for column in list(df.columns) if "Total" in column][0]
    if name is not None:
        name = str(name)
    else:
        name = "Not accounted for"
    df[name] = df_age[df_age_totals_column] - df[df_totals_column]
    df[df_totals_column] = df_age[df_age_totals_column]
    df.rename(columns={df_totals_column:"Total"}, inplace=True)
    return df

### Combining Variables

This function takes an input of 2 data frames of population values. The 2 data frames get passed through the prob_calc function, and then iterated through and combined. The output is a table of combined probabilities.

Data must be input in the form

<table style="border-collapse:collapse;border-spacing:0" class="tg"><thead><tr><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">Region<br></th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">Population</th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">Variable 1</th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">...</th></tr></thead><tbody><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">North East</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">2647013</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:right;vertical-align:top;word-break:normal">2536430</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">...</td></tr><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">North West</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">7417399</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">6885187</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">...</td></tr></tbody></table>

<table style="border-collapse:collapse;border-spacing:0" class="tg"><thead><tr><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">Region<br></th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">Population</th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">Variable 2</th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">...</th></tr></thead><tbody><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">North East</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">2647013</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:right;vertical-align:top;word-break:normal">2536430</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">...</td></tr><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">North West</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">7417399</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">6885187</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">...</td></tr></tbody></table>

and output in the form

<table style="border-collapse:collapse;border-spacing:0" class="tg"><thead><tr><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">Region<br></th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">Population</th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">Probability of Var1 and Var2</th><th style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">...</th></tr></thead><tbody><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">North East</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">2647013</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:right;vertical-align:top;word-break:normal">2536430</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">...</td></tr><tr><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">North West</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">7417399</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">6885187</td><td style="border-color:inherit;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;text-align:left;vertical-align:top;word-break:normal">...</td></tr></tbody></table>

In [ ]:
def combine_probability_tables(inputdata1, inputdata2):
    df1 = prob_calc(inputdata1)
    df2 = prob_calc(inputdata2)

    
    #create lists of characteristics in each table
    df1_names = df1.columns.values.tolist()
    df2_names = df2.columns.values.tolist()
    
    #create a new table to store the combined probabilities, our results
    dfprobability = pd.DataFrame();
    dfprobability["Region"] = df1[df1_names[0]]
    dfprobability["Total Population"] = df1[df1_names[1]]
    
    del df1_names[:2]
    del df2_names[:2]
    
    #nested for loop iterates through columns of both tables and multiplies them together 
    #output is a series which is then added to the reults table
    for col1 in df1_names:
        for col2 in df2_names:
            s = df1[col1] * df2[col2]
            dfprobability = pd.concat([dfprobability,s.rename(col1+" and " +col2)], axis=1)
     
    #output result
    return dfprobability

We also looked at expanding this function to consider more than 2 characteristics


In [1]:
#function takes an input of a list of characteristic codes. assumes data has been imported via import_data or import_all_data functions, 
#assmues geography code column has been removed by calling cleanup_all(data,False,True)
def create_prob_table(list):
    
    count = 1
    if len(list) < 2:
        raise ValueError("Must input more than one characteristic")
    else:
        for i in list:
            if count == 1:
                dfprobability = combineprobabilitytables(data[list[0]], data[list[1]])
            elif count != len(list):
                dfprobability = combineprobabilitytables(dfprobability, data[list[count]])
        
            count+= 1
    

    return dfprobability

### Identifying Select Cells

### Probability Function

Function to calculate probailities as a percent of the total in each region.

Data must be input in the form

| Region     | Population | Variable | ... |
|------------|------------|----------|-----|
| North East | 2647013    |  2536430 | ... |
| North West | 7417399    | 6885187  | ... |

and will be output in the form

| Region     | Population | Probability | ... |
|------------|------------|-------------|-----|
| North East | 2647013    |    0.958223 | ... |
| North West | 7417399    | 0.928248    | ... |  |  |  |

In [2]:
def prob_calc(dataframe):
    probabilities = pd.DataFrame(dataframe)
    total = dataframe.loc[:,dataframe.columns[1]]
    
    count = 2

    # Iterates through columns in dataframe
    for num in range(len(dataframe.columns)-2):
        column = dataframe.loc[:,dataframe.columns[count]]
        cell_count = 0
        # Iterates through items in each column
        for row in column:
            #Divides current cell by the item at the start of the row
            probabilities.loc[:, dataframe.columns[count]].at[cell_count] = row / total[cell_count]
            cell_count += 1
        count += 1
    
    return probabilities

### Correlations in the Data

### Visualising the Data

### Extension - Research how ONS privacy protection exercises have impacted the final results

### Extension - Explore wider utility of the census data for other Hartree Centre projects

### Extension - Consider other open data sources